In [13]:
import ydf
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [6]:
from DataLoader import DataLoader

In [7]:
dataset = DataLoader('../dataset/splitted-animations', 0.8).load_dataset()

Loading dataset


  0%|          | 0/9951 [00:00<?, ?it/s]

100%|██████████| 9951/9951 [02:19<00:00, 71.16it/s]


In [8]:
model = ydf.from_tensorflow_decision_forests(directory="/home/_alk/Documentos/uni/tfg/src/Models/src/models/best_rf_0.8")

In [9]:
num_trees=200
max_depth=15
min_examples=2

In [10]:
model.describe(output_format="notebook")

trial,score,duration,num_trees,max_depth,min_examples
15,0.836516,68.8813,200,15,2
19,0.831491,100.637,200,15,5
24,0.829146,127.149,100,15,2
25,0.823283,133.84,100,15,5
12,0.818425,46.4889,200,15,8
16,0.816918,81.7535,200,15,10
27,0.816583,152.914,150,15,8
18,0.813903,87.5714,50,15,2
14,0.810385,55.3735,100,15,8
3,0.806365,16.8924,50,15,5


In [11]:
model.plot_tree()

In [27]:
model_retrain_learner = ydf.RandomForestLearner(
    num_trees=num_trees,
    max_depth=max_depth,
    min_examples=min_examples,
    task=ydf.Task.CLASSIFICATION,
    label="label",
    random_seed=42
)

train_data, test_data = train_test_split(
            dataset, test_size=0.40, random_state=42)
test_data, val_data = train_test_split(
    test_data, test_size=0.50, random_state=42)
y_train, X_train = zip(*train_data)
y_test, X_test = zip(*test_data)
y_val, X_val = zip(*val_data)

X_train = np.array(X_train, dtype=np.float32)

X_test = np.array(X_test, dtype=np.float32)

X_val = np.array(X_val, dtype=np.float32)


X_reshaped = X_train.reshape(X_train.shape[0], -1)
df_train = pd.DataFrame(
    X_reshaped, columns=[f"feature_{i}" for i in range(X_reshaped.shape[1])])

X_val_reshaped = X_val.reshape(X_val.shape[0], -1)
df_val = pd.DataFrame(
    X_val_reshaped, columns=[f"feature_{i}" for i in range(X_val_reshaped.shape[1])])

X_test_reshaped = X_test.reshape(X_test.shape[0], -1)
df_test = pd.DataFrame(
    X_test_reshaped, columns=[f"feature_{i}" for i in range(X_test_reshaped.shape[1])])


# Ensure 'label' column is added and is of string type
df_train['label'] = y_train
df_val['label'] = y_val
df_test['label'] = y_test


model_retrained = model_retrain_learner.train(df_train)


Train model on 5970 examples
Model trained in 0:00:14.859856


In [28]:
model_retrained.describe(output_format="notebook")

In [29]:
model_retrained.to_tensorflow_saved_model(
                "./Final_Model", mode="tf", servo_api=True)

I0000 00:00:1746777044.635630   31960 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5360 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1
2025-05-09 09:50:44.675159: I tensorflow_decision_forests/tensorflow/ops/inference/kernel.cc:1206] Loading model from path /tmp/tmpaqkph3jc/tmph81uc8e_/ with prefix 717ca8b9_
I0000 00:00:1746777045.531900   31960 decision_forest.cc:808] Model loaded with 200 root(s), 189780 node(s), and 600 input feature(s).
I0000 00:00:1746777045.535321   31960 abstract_model.cc:1439] Engine "RandomForestGeneric" built
2025-05-09 09:50:45.535457: I tensorflow_decision_forests/tensorflow/ops/inference/kernel.cc:1035] Use fast generic engine
